In [1]:
%matplotlib ipympl
from qick import QickSoc, AveragerProgram, RAveragerProgram, deg2reg
from qick.helpers import gauss
from tqdm import tqdm_notebook as tqdm
from matplotlib import pyplot as plt
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from helpers.dataTransfer import saveFullDataToRemote, saveDataToRemote
from Hatlab_DataProcessing.analyzer import qubit_functions_rot as qfr


# Data Saving Info

In [3]:
remotePath = "C:/Users/hatla/Data"
sampleName = "PanfluteA"

# Configuring the channels

The data in this notebook was taken with a wiring diagram similar to the above. The QICK has eight RF DACs which can be configured in multiple ways. Using the QICK, two RF DACs are needed to control the qubit and its readout resonator, respectively. Also, the qubit control pulses (< 6 GHz) were directly synthesized without the use of an analog mixer. Note that the QICK RF board was not used for these measurements, only the ZCU111 evaluation board plus the analog components specified in the diagram.

In [2]:
# Load bitstream with custom overlay
soc = QickSoc()
soc.set_nyquist(ch=7,nqz=1) #Readout_I
soc.set_nyquist(ch=6,nqz=1) #Readout_Q
soc.set_nyquist(ch=3,nqz=2) #Qubit
hw_cfg={"res_ch_I":6,
        "res_ch_Q":7,
        "qubit_ch":3,
       }

# TOF Measurement

Measures the "time of flight" of the measurement pulse. The time of flight is the time at which the measurement pulse appears in the ADC buffer. We only want to start capturing data from this point in time onwards. We store this time of flight in the parameter <code> readout_cfg["adc_trig_offset"] </code>.

In [3]:
class LoopbackProgram(AveragerProgram):
    def initialize(self):
        cfg=self.cfg   
        cfg["adc_lengths"]=[cfg["readout_length"]]*2          #add length of adc acquisition to config
        cfg["adc_freqs"]=[soc.adcfreq(cfg["res_freq"])]*2   #add frequency of adc dds to config
        
        self.res_gain_I = int(cfg["res_gain"])
        self.res_gain_Q = int(cfg["res_gain"] * cfg["IQScale"])
        self.res_phase_I = deg2reg(cfg["res_phase"])
        self.res_phase_Q = deg2reg(cfg["skewPhase"]+cfg["res_phase"])
        self.res_length = cfg["res_length"]
        self.res_freq = soc.freq2reg(soc.adcfreq(cfg["res_freq"]))  # convert frequency to dac frequency (ensuring it is an available adc frequency)

        self.add_pulse(ch=cfg["res_ch_I"], name="measure", style="const", length=self.res_length)  #add a constant pulse to the pulse library
        self.add_pulse(ch=cfg["res_ch_Q"], name="measure", style="const", length=self.res_length)  #add a constant pulse to the pulse library
        
        self.waiti(0, 200)  # give processor some time to configure pulses
    
    def body(self):
        cfg=self.cfg   
        self.trigger_adc(adc1=1, adc2=1, adc_trig_offset=cfg["adc_trig_offset"], t=0)  # trigger the adc acquisition
        self.pulse(ch=cfg["res_ch_I"], name="measure", freq=self.res_freq, phase=self.res_phase_I, gain=self.res_gain_I, length=self.res_length, t=0, play=True) 
        self.pulse(ch=cfg["res_ch_Q"], name="measure", freq=self.res_freq, phase=self.res_phase_Q, gain=self.res_gain_Q, length=self.res_length, t=0, play=True) 

        self.sync_all(soc.us2cycles(self.cfg["relax_delay"]))  # sync all channels

readout_cfg={
    "reps":1, # --Fixed
    "adc_trig_offset": 380, # [clock ticks]
    "readout_length":800, # [clock ticks]
    
    "res_freq": 90, # [MHz]
    "res_gain": 15000, # [DAC units]
    "res_length": 500, # [clock ticks]
    "res_phase" : 0, # [deg]
    "skewPhase":83, # [Degrees]
    "IQScale":1.03,
    
    "soft_avgs": 1000,
    "relax_delay": 10 # [us]
}

config={**hw_cfg, **readout_cfg}
prog =LoopbackProgram(config)
adc1, adc2 = prog.acquire_decimated(soc, load_pulses=True, progress=True, debug=False)

# Plot results.
ax1=plt.subplot(111, title=f"Averages = {config['soft_avgs']}", xlabel="Clock ticks", ylabel="Transmission (adc levels)")
ax1.plot(adc1[0], label="I value; ADC 0")
ax1.plot(adc1[1], label="Q value; ADC 0")
ax1.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

# IQ Histogram

In [17]:
readout_cfg={
    "reps":20000, # 
    "adc_trig_offset": 380, # [clock ticks]
    "readout_length":800, # [clock ticks]
    
    "res_freq": 90, # [MHz]
    "res_gain": 15000, # [DAC units]
    "res_length": 500, # [clock ticks]
    "res_phase" : 0, # [deg]
    "skewPhase":83, # [Degrees]
    "IQScale":1.03,
    
    "rounds": 1,
    "relax_delay": 20 #[us]
}

config={**hw_cfg, **readout_cfg}
prog =LoopbackProgram(config)
avgi, avgq = prog.acquire(soc, load_pulses=True,progress=True, debug=False)

# Plot results.
fig, ax = plt.subplots()
hist=ax.hist2d(prog.di_buf[0],prog.dq_buf[0], bins=101, range=[[-400,400], [-400,400]])
ax.set_aspect(1)
fig.colorbar(hist[3])
plt.show()

# fig, ax = plt.subplots()
# hist=ax.plot(prog.di_buf[0],prog.dq_buf[0], ".")
# ax.set_aspect(1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

# Pulse Probe (qubit) spectroscopy

Measures the qubit frequency f_ge. We store this parameter here: <code> qubit_cfg["f_ge"] </code>.

This program uses the RAveragerProgram class, which allows you to sweep a parameter directly on the processor rather than in a Python loop as in the above resonator spectroscopy example. Because the whole sweep is done on the processor there is less downtime (especially for fast experiments).

In [12]:
class PulseProbeSpectroscopyProgram(RAveragerProgram):
    def initialize(self):
        cfg=self.cfg
        self.cfg["adc_lengths"]=[self.cfg["readout_length"]]*2   #copy over adc acquisition parameters
        self.cfg["adc_freqs"]=[soc.adcfreq(cfg["res_freq"])]*2
        
        self.res_gain_I = int(cfg["res_gain"])
        self.res_gain_Q = int(cfg["res_gain"] * cfg["IQScale"])
        self.res_phase_I = deg2reg(cfg["res_phase"])
        self.res_phase_Q = deg2reg(cfg["skewPhase"]+cfg["res_phase"])
        self.res_length = cfg["res_length"]
        self.res_freq = soc.freq2reg(soc.adcfreq(cfg["res_freq"]))  # convert frequency to dac frequency (ensuring it is an available adc frequency)

        self.qubit_gain = int(cfg["qubit_gain"])
   
        
        self.q_rp=self.ch_page(self.cfg["qubit_ch"])     # get register page for qubit_ch
        self.r_freq=self.sreg(cfg["qubit_ch"], "freq")   # get frequency register for qubit_ch    

        
        # add qubit and readout pulses to respective channels
        self.add_pulse(ch=self.cfg["qubit_ch"], name="qubit",style="const", length=self.cfg["probe_length"])
        self.add_pulse(ch=cfg["res_ch_I"], name="measure", style="const", length=self.res_length)  #add a constant pulse to the pulse library
        self.add_pulse(ch=cfg["res_ch_Q"], name="measure", style="const", length=self.res_length)  #add a constant pulse to the pulse library

        self.f_start = soc.freq2reg(cfg["start"])  # get start/step frequencies
        self.f_step = soc.freq2reg(cfg["step"])
        
        # pre-initialize pulses
        self.pulse(ch=cfg["qubit_ch"], name="qubit", phase=0, freq=self.f_start, gain=cfg["qubit_gain"], play=False)
        self.sync_all(soc.us2cycles(1))
    
    def body(self):
        cfg=self.cfg
        self.pulse(ch=self.cfg["qubit_ch"], play=True)  #play probe pulse        
        self.sync_all(soc.us2cycles(0.05)) # align channels and wait 50ns
        self.trigger_adc(adc1=1, adc2=1, adc_trig_offset=cfg["adc_trig_offset"])  # trigger the adc acquisition
        self.pulse(ch=cfg["res_ch_I"], name="measure", freq=self.res_freq, phase=self.res_phase_I, gain=self.res_gain_I, play=True) 
        self.pulse(ch=cfg["res_ch_Q"], name="measure", freq=self.res_freq, phase=self.res_phase_Q, gain=self.res_gain_Q, play=True) 
        self.waiti(0, cfg["adc_trig_offset"] + cfg["readout_length"]) 
        self.sync_all(soc.us2cycles(self.cfg["relax_delay"]))  # wait for qubit to relax
    
    def update(self):
        self.mathi(self.q_rp, self.r_freq, self.r_freq, '+', self.f_step) # update frequency list index

In [16]:
expt_cfg={"start":4860, # MHz
          "step":0.5, 
          "expts":100, 
          "reps": 500,
          "rounds":1,
          "probe_length":soc.us2cycles(2), 
          "qubit_gain":5000,
          "relax_delay": 200 #[us]
         }

config={**hw_cfg,**readout_cfg,**expt_cfg} #combine configs

qspec=PulseProbeSpectroscopyProgram(cfg=config)
expt_pts, avgi, avgq = qspec.acquire(soc, load_pulses=True,progress=True, debug=False)

plt.figure()
plt.subplot(111,title="Qubit Spectroscopy", xlabel="Qubit Frequency (GHz)", ylabel="Qubit I")
plt.plot(expt_pts, avgi[0][0],'o-')
plt.plot(expt_pts, avgq[0][0],'o-')
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

# Amplitude Rabi

Measures Rabi oscillations by sweeping over the amplitude of the qubit drive pulse, which is an $\approx$ 100 ns ($\sigma$ $\approx$ 25 ns) long Gaussian pulse. We measure the $\pi$ and $\pi$/2 pulse amplitudes which are used in subsequent experiments to control the qubit. We store these parameters here: <code> qubit_cfg["pi_gain"] </code> and <code> qubit_cfg["pi2_gain"] </code>

In [18]:
class AmplitudeRabiProgram(RAveragerProgram):
    def initialize(self):
        cfg=self.cfg
        self.cfg["adc_lengths"]=[self.cfg["readout_length"]]*2   #copy over adc acquisition parameters
        self.cfg["adc_freqs"]=[soc.adcfreq(cfg["res_freq"])]*2
        
        
        self.res_gain_I = int(cfg["res_gain"])
        self.res_gain_Q = int(cfg["res_gain"] * cfg["IQScale"])
        self.res_phase_I = deg2reg(cfg["res_phase"])
        self.res_phase_Q = deg2reg(cfg["skewPhase"]+cfg["res_phase"])
        self.res_length = cfg["res_length"]
        self.res_freq = soc.freq2reg(soc.adcfreq(cfg["res_freq"]))  # convert frequency to dac frequency (ensuring it is an available adc frequency)
        
        self.qubit_freq=soc.freq2reg(cfg["qubit_freq"])
                
        self.q_rp=self.ch_page(self.cfg["qubit_ch"])     # get register page for qubit_ch
        self.r_gain=self.sreg(cfg["qubit_ch"], "gain")   # get gain register for qubit_ch    
        
        # add qubit and readout pulses to respective channels
        self.add_pulse(ch=self.cfg["qubit_ch"], name="qubit", style="arb", 
                       idata=gauss(mu=cfg["sigma"]*16*4/2,si=cfg["sigma"]*16,length=4*cfg["sigma"]*16,maxv=2**15-1) )
        
        self.add_pulse(ch=cfg["res_ch_I"], name="measure", style="const", length=self.res_length)  #add a constant pulse to the pulse library
        self.add_pulse(ch=cfg["res_ch_Q"], name="measure", style="const", length=self.res_length)  #add a constant pulse to the pulse library

        
        # pre-initialize pulses
        self.pulse(ch=cfg["qubit_ch"], name="qubit", phase=0, freq=self.qubit_freq, gain=cfg["start"], play=False)    
        self.sync_all(soc.us2cycles(1))
    
    def body(self):   
        cfg=self.cfg
        self.pulse(ch=self.cfg["qubit_ch"], play=True, phase=deg2reg(90))  #play gaussian pulse
        self.sync_all(soc.us2cycles(0.05)) # align channels and wait 50ns
        self.trigger_adc(adc1=1, adc2=1, adc_trig_offset=cfg["adc_trig_offset"])  # trigger the adc acquisition
        self.pulse(ch=cfg["res_ch_I"], name="measure", freq=self.res_freq, phase=self.res_phase_I, gain=self.res_gain_I, play=True) 
        self.pulse(ch=cfg["res_ch_Q"], name="measure", freq=self.res_freq, phase=self.res_phase_Q, gain=self.res_gain_Q, play=True) 
        self.waiti(0, cfg["adc_trig_offset"] + cfg["readout_length"]) 
        self.sync_all(soc.us2cycles(self.cfg["relax_delay"]))  # wait for qubit to relax
    
    def update(self):
        self.mathi(self.q_rp, self.r_gain, self.r_gain, '+', self.cfg["step"]) # update gain of the Gaussian pi pulse

In [19]:
qubit_cfg={
    "sigma": soc.us2cycles(0.07),
    "qubit_freq":4879
}

expt_cfg={
    "start":-30000, 
    "step":200, 
    "expts":300, 
    "reps": 200,
    "relax_delay":200
       }

config={**hw_cfg,**readout_cfg,**qubit_cfg,**expt_cfg} #combine configs

rabi=AmplitudeRabiProgram(cfg=config)
x_pts, avgi, avgq  = rabi.acquire(soc,load_pulses=True,progress=True, debug=False)

plt.figure()
plt.subplot(111, title= f"Amplitude Rabi, $\sigma={soc.cycles2us(config['sigma'])*1000}$ ns", xlabel="Gain", ylabel="Qubit Population" )
plt.plot(x_pts,avgi[0][0],'o-')
plt.plot(x_pts,avgq[0][0],'o-')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

In [20]:
piPul = qfr.PiPulseTuneUp(x_pts, avgi[0][0]+1j*avgq[0][0])
piResult = piPul.run()
piResult.plot()

# # ----- save data to pc -------- 
# data_temp = {"x_data":x_pts, "i_data": avgi[0][0], "q_data": avgq[0][0]}
# saveDataToRemote(data_temp, sampleName+"_pipulse", remotePath)

Pi pulse amp:17118.22 DAC, Pi/2 pulse amp:8480.422 DAC


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

# T1 

Measures the qubit T1 characteristic decay time by preparing the qubit in its excited state with a $\pi$ pulse followed by a variable delay ($\tau$) before measuring the qubit's state. This exponential decay curve can be fitted in post-processing to obtain the qubit T1 time.

In [21]:
class T1Program(RAveragerProgram):
    def __init__(self,cfg):
        AveragerProgram.__init__(self,cfg)       
        
    def initialize(self):
        cfg=self.cfg               
        self.cfg["adc_lengths"]=[self.cfg["readout_length"]]*2   #copy over adc acquisition parameters
        self.cfg["adc_freqs"]=[soc.adcfreq(cfg["res_freq"])]*2
        
        self.res_gain_I = int(cfg["res_gain"])
        self.res_gain_Q = int(cfg["res_gain"] * cfg["IQScale"])
        self.res_phase_I = deg2reg(cfg["res_phase"])
        self.res_phase_Q = deg2reg(cfg["skewPhase"]+cfg["res_phase"])
        self.res_length = cfg["res_length"]
        self.res_freq = soc.freq2reg(soc.adcfreq(cfg["res_freq"]))  # convert frequency to dac frequency (ensuring it is an available adc frequency)
        
        self.qubit_freq=soc.freq2reg(cfg["qubit_freq"])
        
                                
        self.q_rp=self.ch_page(self.cfg["qubit_ch"])     # get register page for qubit_ch
        self.r_wait = 3
        self.regwi(self.q_rp, self.r_wait, soc.us2cycles(cfg["start"]))
        
        
        # add qubit and readout pulses to respective channels
        self.add_pulse(ch=self.cfg["qubit_ch"], name="qubit", style="arb", 
                       idata=gauss(mu=cfg["sigma"]*16*4/2,si=cfg["sigma"]*16,length=4*cfg["sigma"]*16,maxv=2**15-1) )
        
        self.add_pulse(ch=cfg["res_ch_I"], name="measure", style="const", length=self.res_length)  #add a constant pulse to the pulse library
        self.add_pulse(ch=cfg["res_ch_Q"], name="measure", style="const", length=self.res_length)  #add a constant pulse to the pulse library

               
        # pre-initialize pulses
        self.pulse(ch=cfg["qubit_ch"], name="qubit", phase=0, freq=self.qubit_freq, gain=cfg["pi_gain"], play=False)
    
        self.sync_all(soc.us2cycles(1))
    
    def body(self):
        cfg = self.cfg
        self.pulse(ch=self.cfg["qubit_ch"], play=True)  #play pi pulse
        self.sync_all()
        self.sync(self.q_rp,self.r_wait)
        self.trigger_adc(adc1=1, adc2=1, adc_trig_offset=cfg["adc_trig_offset"])  # trigger the adc acquisition
        self.pulse(ch=cfg["res_ch_I"], name="measure", freq=self.res_freq, phase=self.res_phase_I, gain=self.res_gain_I, play=True) 
        self.pulse(ch=cfg["res_ch_Q"], name="measure", freq=self.res_freq, phase=self.res_phase_Q, gain=self.res_gain_Q, play=True) 
        self.waiti(0, cfg["adc_trig_offset"] + cfg["readout_length"]) 
        self.sync_all(soc.us2cycles(self.cfg["relax_delay"]))  # wait for qubit to relax

    
    def update(self):
        self.mathi(self.q_rp, self.r_wait, self.r_wait, '+', soc.us2cycles(self.cfg["step"])) # update frequency list index

In [22]:
qubit_cfg={
    "sigma": soc.us2cycles(0.07),
    "qubit_freq":4879,
    "pi_gain": -17041,
    "pi2_gain":-8492
}


expt_cfg={ 
    "start":0, # [us]
    "step":2,  # [us]
    "expts":200, 
    "reps": 2000,     
    "relax_delay":500,
       }

config={**hw_cfg,**readout_cfg,**qubit_cfg,**expt_cfg} #combine configs
t1p=T1Program(cfg=config)
x_pts, avgi, avgq = t1p.acquire(soc, load_pulses=True, progress=True, debug=False)

plt.figure()
plt.subplot(111, title="T1 Experiment", xlabel=f"Time ($\mu s$)", ylabel="Qubit Population")
plt.plot(x_pts,avgi[0][0],'o-')
plt.plot(x_pts,avgq[0][0],'o-')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

In [23]:
t1Decay = qfr.T1Decay(x_pts, avgi[0][0]+1j*avgq[0][0])
t1Result = t1Decay.run(piResult.rot_result)
t1Result.plot()

# # ----- save data to pc -------- 
# data_temp = {"x_data":x_pts, "i_data": avgi[0][0], "q_data": avgq[0][0]}
# saveDataToRemote(data_temp, sampleName+"_t1", remotePath)

tau is 88.86 us


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

# Ramsey Fringe Experiment

Measures the qubit T2 Ramsey characteristic decay time by preparing the qubit in a superposition state with a  $\frac{\pi}{2}$ pulse followed by a variable delay ($\tau$) before applying another $\frac{\pi}{2}$ pulse with a phase advanced by $\Delta \phi = \omega * \tau$, where $\omega$ is the Ramsey frequency. This exponential decay envelope can be fitted in post-processing to obtain the qubit T2 Ramsey time.

In [ ]:
class RamseyProgram(RAveragerProgram):
    def __init__(self,cfg):
        AveragerProgram.__init__(self,cfg)       
        
    def initialize(self):
        cfg=self.cfg               
        self.cfg["adc_lengths"]=[self.cfg["readout_length"]]*2   #copy over adc acquisition parameters
        self.cfg["adc_freqs"]=[soc.adcfreq(cfg["res_freq"])]*2
        
        self.res_gain_I = int(cfg["res_gain"])
        self.res_gain_Q = int(cfg["res_gain"] * cfg["IQScale"])
        self.res_phase_I = deg2reg(cfg["res_phase"])
        self.res_phase_Q = deg2reg(cfg["skewPhase"]+cfg["res_phase"])
        self.res_length = cfg["res_length"]
        self.res_freq = soc.freq2reg(soc.adcfreq(cfg["res_freq"]))  # convert frequency to dac frequency (ensuring it is an available adc frequency)
             
        self.qubit_freq=soc.freq2reg(cfg["qubit_freq"])
    
        
        self.q_rp=self.ch_page(self.cfg["qubit_ch"])     # get register page for qubit_ch
        self.r_wait = 3
        self.regwi(self.q_rp, self.r_wait, soc.us2cycles(cfg["start"]))
        

        # add qubit and readout pulses to respective channels
        self.add_pulse(ch=self.cfg["qubit_ch"], name="qubit", style="arb", 
                       idata=gauss(mu=cfg["sigma"]*16*4/2,si=cfg["sigma"]*16,length=4*cfg["sigma"]*16,maxv=2**15-1) )
        
        self.add_pulse(ch=cfg["res_ch_I"], name="measure", style="const", length=self.res_length)  #add a constant pulse to the pulse library
        self.add_pulse(ch=cfg["res_ch_Q"], name="measure", style="const", length=self.res_length)  #add a constant pulse to the pulse library

        
        # pre-initialize pulses
        self.pulse(ch=cfg["qubit_ch"], name="qubit", phase=0, freq=self.qubit_freq, gain=cfg["pi2_gain"], play=False)
    
        self.sync_all(soc.us2cycles(1))
    
    def body(self):
        cfg = self.cfg
        
        self.pulse(ch=self.cfg["qubit_ch"], play=True)  #play probe pulse
        self.sync_all()
        self.sync(self.q_rp,self.r_wait)
        self.pulse(ch=self.cfg["qubit_ch"], play=True)  #play probe pulse
        self.sync_all(soc.us2cycles(0.05))
        
        self.trigger_adc(adc1=1, adc2=1, adc_trig_offset=cfg["adc_trig_offset"])  # trigger the adc acquisition
        self.pulse(ch=cfg["res_ch_I"], name="measure", freq=self.res_freq, phase=self.res_phase_I, gain=self.res_gain_I, play=True) 
        self.pulse(ch=cfg["res_ch_Q"], name="measure", freq=self.res_freq, phase=self.res_phase_Q, gain=self.res_gain_Q, play=True) 
        self.waiti(0, cfg["adc_trig_offset"] + cfg["readout_length"]) 
        self.sync_all(soc.us2cycles(self.cfg["relax_delay"]))  # wait for qubit to relax
        
    
    def update(self):
        self.mathi(self.q_rp, self.r_wait, self.r_wait, '+',  soc.us2cycles(self.cfg["step"])) # update the time between two π/2 pulses


In [ ]:
expt_cfg={
    "start":0,  # [us]
    "step":1, # [us]
    "expts":100,
    "reps": 1000, 
    "rounds": 1,
    "relax_delay":400 # [us]
       }

qubit_cfg={
    "sigma": soc.us2cycles(0.07),
    "qubit_freq":4878.8926338,
    "pi_gain": -17041,
    "pi2_gain":-8492
}

config={**hw_cfg,**readout_cfg,**qubit_cfg,**expt_cfg} #combine configs
t2p=RamseyProgram(cfg=config)
x_pts, avgi, avgq= t2p.acquire(soc, load_pulses=True,progress=True, debug=False)

plt.figure()
plt.subplot(111, title="Ramsey Fringe Experiment", xlabel="Delay time ($\mu$s)", ylabel="Qubit Population")
plt.plot(x_pts,avgi[0][0],'o-')
plt.plot(x_pts,avgq[0][0],'o-')

In [ ]:
t2Ramsey = qfr.T2Ramsey(x_pts, avgi[0][0]+1j*avgq[0][0])
t2rResult = t2Ramsey.run(piResult.rot_result)
t2rResult.plot()

# # ----- save data to pc -------- 
# data_temp = {"x_data":x_pts, "i_data": avgi[0][0], "q_data": avgq[0][0]}
# saveDataToRemote(data_temp, sampleName+"_t2R", remotePath)


# T2 Echo Experiment

In [ ]:
class T2EProgram(RAveragerProgram):
    def __init__(self,cfg):
        AveragerProgram.__init__(self,cfg)       
        
    def initialize(self):
        cfg=self.cfg               
        self.cfg["adc_lengths"]=[self.cfg["readout_length"]]*2   #copy over adc acquisition parameters
        self.cfg["adc_freqs"]=[soc.adcfreq(cfg["res_freq"])]*2
        
        self.res_gain_I = int(cfg["res_gain"])
        self.res_gain_Q = int(cfg["res_gain"] * cfg["IQScale"])
        self.res_phase_I = deg2reg(cfg["res_phase"])
        self.res_phase_Q = deg2reg(cfg["skewPhase"]+cfg["res_phase"])
        self.res_length = cfg["res_length"]
        self.res_freq = soc.freq2reg(soc.adcfreq(cfg["res_freq"]))  # convert frequency to dac frequency (ensuring it is an available adc frequency)
             
        self.qubit_freq=soc.freq2reg(cfg["qubit_freq"])
    
        
        self.q_rp=self.ch_page(self.cfg["qubit_ch"])     # get register page for qubit_ch
        self.r_wait = 3
        self.regwi(self.q_rp, self.r_wait, soc.us2cycles(cfg["start"]/2))
        

        # add qubit and readout pulses to respective channels
        self.add_pulse(ch=self.cfg["qubit_ch"], name="qubit", style="arb", 
                       idata=gauss(mu=cfg["sigma"]*16*4/2,si=cfg["sigma"]*16,length=4*cfg["sigma"]*16,maxv=2**15-1) )

        
        self.add_pulse(ch=cfg["res_ch_I"], name="measure", style="const", length=self.res_length)  #add a constant pulse to the pulse library
        self.add_pulse(ch=cfg["res_ch_Q"], name="measure", style="const", length=self.res_length)  #add a constant pulse to the pulse library

        
        # pre-initialize pulses
        self.pulse(ch=cfg["qubit_ch"], name="qubit", phase=0, freq=self.qubit_freq, gain=cfg["pi2_gain"], play=False)
    
        self.sync_all(soc.us2cycles(1))
    
    def body(self):
        cfg = self.cfg
        
        self.pulse(ch=self.cfg["qubit_ch"], play=True)  #play probe pulse
        self.sync_all()
        self.sync(self.q_rp,self.r_wait)
        self.pulse(ch=self.cfg["qubit_ch"], gain=cfg["pi_gain"], play=True)  #play probe pulse
        self.sync_all()
        self.sync(self.q_rp,self.r_wait)
        self.pulse(ch=self.cfg["qubit_ch"], gain=cfg["pi2_gain"], play=True)  #play probe pulse
        
        self.sync_all(soc.us2cycles(0.05))
        
        self.trigger_adc(adc1=1, adc2=1, adc_trig_offset=cfg["adc_trig_offset"])  # trigger the adc acquisition
        self.pulse(ch=cfg["res_ch_I"], name="measure", freq=self.res_freq, phase=self.res_phase_I, gain=self.res_gain_I, play=True) 
        self.pulse(ch=cfg["res_ch_Q"], name="measure", freq=self.res_freq, phase=self.res_phase_Q, gain=self.res_gain_Q, play=True) 
        self.waiti(0, cfg["adc_trig_offset"] + cfg["readout_length"]) 
        self.sync_all(soc.us2cycles(self.cfg["relax_delay"]))  # wait for qubit to relax
        
    
    def update(self):
        self.mathi(self.q_rp, self.r_wait, self.r_wait, '+',  soc.us2cycles(self.cfg["step"]/2)) # update the time between two π/2 pulses


In [ ]:
expt_cfg={
    "start":0,  # [us]
    "step": 1,  # [us]
    "expts":200,
    "reps": 2000, 
    "rounds": 1,
    "relax_delay":400 # [us]
       }

qubit_cfg={
    "sigma": soc.us2cycles(0.07),
    "qubit_freq":4878.8926338,
    "pi_gain": -17041,
    "pi2_gain":-8492
}

config={**hw_cfg,**readout_cfg,**qubit_cfg,**expt_cfg} #combine configs
t2ep=T2EProgram(cfg=config)
x_pts, avgi, avgq= t2ep.acquire(soc, load_pulses=True,progress=True, debug=False)

plt.figure()
plt.subplot(111, title="T2 echo Experiment", xlabel="Delay time ($\mu$s)", ylabel="Qubit Population")
plt.plot(x_pts,avgi[0][0],'o-')
plt.plot(x_pts,avgq[0][0],'o-')

In [ ]:
t2EDecay = qfr.T1Decay(x_pts, avgi[0][0]+1j*avgq[0][0])
t2EResult = t2EDecay.run(piResult.rot_result)
t2EResult.plot()

# # ----- save data to pc -------- 
# data_temp = {"x_data":x_pts, "i_data": avgi[0][0], "q_data": avgq[0][0]}
# saveDataToRemote(data_temp, sampleName+"_t2E", remotePath)